In [1]:
########### IMPORTING THE REQURIED LIBRARIES ###########

from bs4 import BeautifulSoup as soup
import requests
from random import choice
from terminaltables import AsciiTable

In [2]:
WORLD_URL = "https://www.worldometers.info/coronavirus/#countries"
MAX_TIMEOUT = 3

In [3]:
######## DECLARING THE CLASS FOR SCRAPING SSLPROXIES.COM ########

class _proxy:

	PROXY_URL = 'https://sslproxies.org/'
	page = None

	def __init__( self ):
		self.page = soup( requests.get( self.PROXY_URL ).text, 'lxml' )
    
	def getSSLProxyDictionary( self ):
 
		return {
		'https': choice( list( map( lambda x:x[ 0 ] + ':' + x[ 1 ], list( zip( map( lambda x:x.text, self.page.find( 'table' ).findAll( 'td' )[ ::8 ] ), map( lambda x:x.text, self.page.find( 'table' ).findAll( 'td' )[ 1::8 ] ) ) ) ) ) )
		}

In [4]:
proxy = _proxy()

def _loadDataByIPRotaion( url ):
	count = 0
	response = None
    
	while count < 10:
		try:
			proxyDictionary = proxy.getSSLProxyDictionary()

			print( 'Retry {} Using Proxy : {}'.format( count, proxyDictionary ) )

			response = requests.get( url, proxies = proxyDictionary, timeout = MAX_TIMEOUT )
			break
		except:
			pass

		count=count+1

	return response

In [5]:
try:
    resp = requests.get( WORLD_URL, timeout = MAX_TIMEOUT )
    page = soup( resp.text, 'lxml' ) 
except:
    resp = _loadDataByIPRotaion( WORLD_URL )
    page = soup( resp.text, 'lxml' )

In [6]:
table = page.find( "table",{
   "id": "main_table_countries_today" 
} )

In [7]:
total_cases = page.findAll( "div", {
    "id": "maincounter-wrap"
} )[ 0 ].div.text.strip()
total_cases

'446,703'

In [8]:
total_deaths = page.findAll( "div", {
    "id": "maincounter-wrap"
} )[ 1 ].div.text.strip()
total_deaths

'19,805'

In [9]:
total_recovered = page.findAll( "div", {
    "id": "maincounter-wrap"
} )[ 2 ].div.text.strip()
total_recovered

'112,050'

In [10]:
table_heading = [ item.text.strip() for item in table.thead.tr if item != "\n" ]

In [11]:
table_content = []
for rows in table.tbody:
    data = [ item.text.strip() for item in rows if item != "\n" ]
    if data:
        table_content.append( data[ : -2 ] )

In [12]:
len(table_content)

197

In [13]:
table_content.insert( 0, table_heading[ : -2 ] )

In [14]:
table = AsciiTable(table_content)
print( table.table )

+------------------------+------------+----------+-------------+-----------+----------------+-------------+------------------+
| Country,Other          | TotalCases | NewCases | TotalDeaths | NewDeaths | TotalRecovered | ActiveCases | Serious,Critical |
+------------------------+------------+----------+-------------+-----------+----------------+-------------+------------------+
| China                  | 81,218     | +47      | 3,281       | +4        | 73,650         | 4,287       | 1,399            |
| Italy                  | 69,176     |          | 6,820       |           | 8,326          | 54,030      | 3,393            |
| USA                    | 60,642     | +5,786   | 817         | +37       | 379            | 59,446      | 1,307            |
| Spain                  | 47,610     | +5,552   | 3,434       | +443      | 5,367          | 38,809      | 2,636            |
| Germany                | 35,740     | +2,749   | 186         | +27       | 3,540          | 32,014      | 23 

In [15]:
import sys

In [16]:
INDIA_URL = "https://www.mohfw.gov.in/"

In [17]:
try:
    resp = requests.get( INDIA_URL, timeout = MAX_TIMEOUT )
    page = soup( resp.text, 'lxml' ) 
except:
    resp = _loadDataByIPRotaion( INDIA_URL )
    page = soup( resp.text, 'lxml' )

In [31]:
table = page.findAll( "div",{
   "class": "table-responsive" 
} )[ 7 ]

In [43]:
total_cases = int( page.findAll( "div",{
   "class": "table-responsive" 
} )[ 7 ].tbody.findAll( "tr" )[ -1 : ][ 0 ].findAll( "td" )[ 1 ].text.strip() )

total_cases += int( page.findAll( "div",{
   "class": "table-responsive" 
} )[ 7 ].tbody.findAll( "tr" )[ -1 : ][ 0 ].findAll( "td" )[ 2 ].text.strip() )

total_cases

606

In [19]:
table_heading = [ item.text.strip() for item in table.thead.tr if item != "\n" ]

In [20]:
table_heading

['S. No.',
 'Name of State / UT',
 'Total Confirmed cases (Indian National)',
 'Total Confirmed cases ( Foreign National )',
 'Cured/Discharged/Migrated',
 'Death']

In [21]:
table_content = []
for rows in table.tbody:
    data = [ item.text.strip() for item in rows if item != "\n" ]
    if data:
        table_content.append( data[ : -2 ] )

In [22]:
table_content.insert( 0, table_heading[ : -2 ] )

In [23]:
table = AsciiTable(table_content)
print( table.table )

+------------------------------------------+--------------------+-----------------------------------------+--------------------------------------------+
| S. No.                                   | Name of State / UT | Total Confirmed cases (Indian National) | Total Confirmed cases ( Foreign National ) |
+------------------------------------------+--------------------+-----------------------------------------+--------------------------------------------+
| 1                                        | Andhra Pradesh     | 9                                       | 0                                          |
| 2                                        | Bihar              | 4                                       | 0                                          |
| 3                                        | Chhattisgarh       | 1                                       | 0                                          |
| 4                                        | Delhi              | 30              

In [44]:
import scipy